In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps
from FinMind.data import DataLoader
import helper
api = DataLoader()

d:\Practice\stock_trade\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
def print_result(stats):
  stats = stats.loc[[
      'Start',
      'End',
      'Equity Final [$]',      # 最終資產淨值
      'Return [%]',            # 總報酬率
      'Buy & Hold Return [%]', # 買入並持有報酬率（基準）
      'Max. Drawdown [%]',     # 最大回撤
      '# Trades',
      'Win Rate [%]',          # 勝率
      'SQN',                    # 系統品質數字
      '_trades',
      '_strategy'
    ]]
  print('------------------回測表現-----------------')
  print(stats)
  print('------------------交易紀錄-----------------')
  trades = stats['_trades']
  for _, trade in trades.iterrows():
      print(f"Buy date: {trade['EntryTime']}, Sell date: {trade['ExitTime']}")
      print(f"Buy price: {trade['EntryPrice']}, Sell price: {trade['ExitPrice']}, P/L: {trade['ReturnPct']:.2%}")

In [4]:
today = datetime.today() #datetime.strptime('2025-04-30', "%Y-%m-%d")
start_date = (today - relativedelta(months=24)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")
print('start date', start_date)
print('end date', end_date)

ticker = '2348'
df = api.taiwan_stock_daily(
    stock_id=ticker,
    start_date=start_date,
    end_date=end_date
)
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df = df.rename(columns={'max': 'High', 'min': 'Low', 'open': 'Open', 'close': 'Close', 'Trading_Volume': 'Volume'})
df

2025-10-29 10:51:16.821 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanStockPrice, data_id: 2348


start date 2023-10-29
end date 2025-10-29


,stock_id,Volume,Trading_money,Open,High,Low,Close,spread,Trading_turnover
date,,,,,,,,,
2023-10-30,2348,34987,2831649,80.5,81.4,80.5,80.9,0.0,46
2023-10-31,2348,58513,4719557,80.9,81.3,79.7,80.0,-0.9,221
2023-11-01,2348,65870,5328709,81.2,81.6,80.5,81.1,1.1,209
2023-11-02,2348,213546,17572815,81.5,82.6,81.5,82.2,1.1,247
2023-11-03,2348,116839,9647386,82.9,82.9,82.1,82.3,0.1,143
...,...,...,...,...,...,...,...,...,...
2025-10-21,2348,255577,21923771,86.0,86.4,85.0,85.9,0.6,351
2025-10-22,2348,455196,39568294,86.3,87.5,85.8,87.3,1.4,891
2025-10-23,2348,182663,15883777,87.8,87.8,86.3,86.8,-0.5,1171


In [10]:
# helper.EMA_KD.n1 = 20
# helper.EMA_KD.n2 = 60
bt = Backtest(df, helper.EMA_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[20, 50, 60, 100, 120],
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/20 [00:00<?, ?it/s]

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=455: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=457: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)


d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=316: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=255: Broker canceled the relative-sized order due to insufficient margin.


------------------回測表現-----------------
Start                                                  2023-10-30 00:00:00
End                                                    2025-10-28 00:00:00
Equity Final [$]                                              96856.661175
Return [%]                                                       -3.143339
Buy & Hold Return [%]                                                  0.0
Max. Drawdown [%]                                                -5.331868
# Trades                                                                 1
Win Rate [%]                                                           0.0
SQN                                                                    NaN
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                             EMA_KD(n1=20,n2=100)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-12-26 00:00:00, Sell date: 2024-12-27 00:00:00
Buy price: 244.0, S

In [5]:
helper.EMA_VWAP_KD.n1 = 10
helper.EMA_VWAP_KD.n2 = 50
bt = Backtest(df, helper.EMA_VWAP_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/24 [00:00<?, ?it/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=163: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close t

------------------回測表現-----------------
Start                                                  2023-10-30 00:00:00
End                                                    2025-10-28 00:00:00
Equity Final [$]                                              96856.661175
Return [%]                                                       -3.143339
Buy & Hold Return [%]                                                  0.0
Max. Drawdown [%]                                                -5.331868
# Trades                                                                 1
Win Rate [%]                                                           0.0
SQN                                                                    NaN
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                        EMA_VWAP_KD(n1=20,n2=100)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-12-26 00:00:00, Sell date: 2024-12-27 00:00:00
Buy price: 244.0, S

In [8]:
# helper.SmaCross.n1 = 10
# helper.SmaCross.n2 = 100
bt = Backtest(df, helper.SmaCross, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/24 [00:00<?, ?it/s]

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=259: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=311: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=452: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=43: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=143: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtes

------------------回測表現-----------------
Start                                                  2023-10-30 00:00:00
End                                                    2025-10-28 00:00:00
Equity Final [$]                                             130843.869325
Return [%]                                                       30.843869
Buy & Hold Return [%]                                            -2.749141
Max. Drawdown [%]                                               -70.468146
# Trades                                                                 4
Win Rate [%]                                                         100.0
SQN                                                               1.463738
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                            SmaCross(n1=10,n2=20)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-01-03 00:00:00, Sell date: 2024-07-01 00:00:00
Buy price: 92.5, Se

In [5]:
bt = Backtest(df, helper.SMA_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  kd=80,
  maximize='Equity Final [$]'
)
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/24 [00:00<?, ?it/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=43: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=53: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Pract

------------------回測表現-----------------
Start                                                  2023-10-30 00:00:00
End                                                    2025-10-28 00:00:00
Equity Final [$]                                             128471.594185
Return [%]                                                       28.471594
Buy & Hold Return [%]                                            -2.749141
Max. Drawdown [%]                                               -58.083147
# Trades                                                                 3
Win Rate [%]                                                         100.0
SQN                                                               3.206065
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                         SMA_KD(n1=5,n2=20,kd=80)
dtype: object
------------------交易紀錄-----------------
Buy date: 2023-12-28 00:00:00, Sell date: 2024-03-14 00:00:00
Buy price: 92.8, Se

In [14]:
helper.BOLL_KD30.buy_strategy = helper.BuyStrategy.BOLL_KD30
bt = Backtest(df, helper.BOLL_KD30, cash=100_000, commission=.001425)
stats = bt.run()
print_result(stats)

Backtest.run:   0%|          | 0/467 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=134: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=147: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=192: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=217: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=382: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d

------------------回測表現-----------------
Start                                                  2023-10-23 00:00:00
End                                                    2025-10-22 00:00:00
Equity Final [$]                                              87599.816441
Return [%]                                                      -12.400184
Buy & Hold Return [%]                                            11.111111
Max. Drawdown [%]                                               -29.487118
# Trades                                                                 4
Win Rate [%]                                                          25.0
SQN                                                              -0.506367
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-01-19 00:00:00, Sell date: 2024-03-28 00:00:00
Buy price: 50.2, Sell price: 53.8, P/L: 6.88%
Buy date: 2024-08-06 00:00:00, Sell date: 2024-1

C:\Users\evonne.chin\AppData\Local\Temp\ipykernel_31796\1809897457.py:3: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()
